In [ ]:
%load_ext autoreload
%autoreload 2

# White background for matplotlib plots
# %config InlineBackend.print_figure_kwargs={'facecolor': "w"}

import numpy as np
import tensorflow as tf
import torch
import tqdm.notebook as tqdm
import pandas as pd

# import thesis.benchmark as qb

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10, 5]

import seaborn as sns

import gin
gin.enter_interactive_mode()
# %matplotlib widget

In [ ]:
hidden_size = 10
expansion = 4

dense_keras = tf.keras.Sequential(
            [
                tf.keras.layers.Input((hidden_size,)),
                tf.keras.layers.Dense(hidden_size * expansion, activation=tf.nn.relu),
                # tf.keras.layers.Dense(hidden_size, use_bias=False),
            ]
        )

In [ ]:
import tensorflow as tf
from tensorflow.python.keras.engine.base_layer import Layer
from tensorflow.keras.layers import Conv2D
from tensorflow.python.profiler.model_analyzer import profile
from tensorflow.python.profiler.option_builder import ProfileOptionBuilder
print('TensorFlow:', tf.__version__)


# model = tf.keras.applications.ResNet50()
forward_pass = tf.function(
    dense_keras.call,
    input_signature=[tf.TensorSpec(shape=(1, hidden_size))])

graph_info = profile(forward_pass.get_concrete_function().graph,
                        options=ProfileOptionBuilder.float_operation())

flops = graph_info.total_float_ops

In [ ]:
print('Flops: {:,}'.format(flops))

In [ ]:
import plotly.express as px

In [ ]:
df = pd.read_csv("/Volumes/euler/benchmark_data/0606-dense-4/results.csv")
df = df.drop(columns={"Unnamed: 0"})

g = sns.relplot(
    data=df,
    hue="name",
    x="model_i",
    y="inference_time",
    kind="line",
#     ci="sd",
)
# g.set(title="CNN", yscale="log")
plt.show()

# sns.distplot(data=df[df["network_size"] == max(df["network_size"])])

In [ ]:
fig = px.line(df, x="model_i", y="inference_time", color='name')
fig.show()

In [ ]:
df.loc[df["network_size"] == 0]

In [ ]:
df2 = df.loc[df["network_size"] == 0].groupby("name").agg({
    "loss": "mean",
    "inference_time": ["mean", "std"],
})
df2.columns = ['_'.join(ind).strip() for ind in df2.columns.values]

for name, row in df2.iterrows():
    print(dict(row))

In [ ]:
df = pd.read_csv("/Volumes/euler/benchmark_data/0523_cnn.csv")
df = df.drop(columns={"Unnamed: 0"})

g = sns.relplot(
    data=df,
    hue="name",
    x="network_size",
    y="inference_time",
    kind="line",
#     ci="sd",
)
g.set(title="CNN", yscale="log")
plt.show()

sns.distplot(data=df[df["network_size"] == max(df["network_size"])])

In [ ]:
sns.displot(data=df[df["network_size"] == max(df["network_size"])], x="inference_time", hue="name", kind="ecdf")

In [ ]:
df = pd.read_csv("/Volumes/euler/benchmark_data/0520_dense.csv")
df = df.drop(columns={"Unnamed: 0"})

g = sns.relplot(
    data=df,
    hue="name",
    x="network_size",
    y="inference_time",
    kind="line",
#     ci="sd",
)
g.set(title="Dense", yscale="log")

In [ ]:
df = pd.read_csv("/Volumes/euler/benchmark_data/0520_dense_3_layer.csv")
df = df.drop(columns={"Unnamed: 0"})

g = sns.relplot(
    data=df,
    hue="name",
    x="network_size",
    y="inference_time",
    kind="line",
#     ci="sd",
)
g.set(title="Dense - 3 layers", yscale="log")

In [ ]:
df = pd.read_csv("/Volumes/euler/benchmark_data/0520_cnn_1_layer.csv")
df = df.drop(columns={"Unnamed: 0"})

g = sns.relplot(
    data=df,
    hue="name",
    x="network_size",
    y="inference_time",
    kind="line",
#     ci="sd",
)
g.set(title="Dense", yscale="log")

In [ ]:
df = pd.read_csv("/Volumes/euler/benchmark_data/0520_cnn_3_layer.csv")
df = df.drop(columns={"Unnamed: 0"})

g = sns.relplot(
    data=df,
    hue="name",
    x="network_size",
    y="inference_time",
    kind="line",
#     ci="sd",
)
g.set(title="CNN 3 layers", yscale="log")

In [ ]:
df = pd.read_csv("/Volumes/euler/benchmark_data/0520_dilated_cnn_small.csv")
df = df.drop(columns={"Unnamed: 0"})

g = sns.relplot(
    data=df,
    hue="name",
    x="network_size",
    y="inference_time",
    kind="line",
#     ci="sd",
)
g.set(title="Dense", yscale="log")

In [ ]:
df

In [ ]:
df = pd.read_csv("/Volumes/euler/benchmark_data/0520_dilated_cnn_1_layer.csv")
df = df.drop(columns={"Unnamed: 0"})

g = sns.relplot(
    data=df,
    hue="name",
    x="network_size",
    y="inference_time",
    kind="line",
#     ci="sd",
)
g.set(title="Dilated CNN - 1 layer", yscale="log")

In [ ]:
df = pd.read_csv("/Volumes/euler/benchmark_data/0520_dilated_cnn_3_layers.csv")
df = df.drop(columns={"Unnamed: 0"})

g = sns.relplot(
    data=df,
    hue="name",
    x="network_size",
    y="inference_time",
    kind="line",
#     ci="sd",
)
g.set(title="Dilated CNN - 3 layers", yscale="log")

## ----

In [ ]:
import tensorflow_model_optimization as tfmot

size = 32
n_channels = 16

# model = tf.keras.Sequential(
#     [
#         tf.keras.layers.Input((size, size, n_channels)),
#         tf.keras.layers.Conv2D(
#             filters=n_channels,
#             kernel_size=3,
#             activation=tf.nn.relu,
#             padding="same",
#         ),
#         tf.keras.layers.MaxPooling2D(size),
#     ]
# )

model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(28, 28)),
    tf.keras.layers.Reshape(target_shape=(28, 28, 1)),
    tf.keras.layers.Conv2D(filters=12, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10)
])

model.compile(
      loss=tf.keras.losses.MeanSquaredError(),
      optimizer='adam',
      metrics=['accuracy']
)


# pruning_params = {'block_size': [1, 16]}
model_for_pruning = tfmot.sparsity.keras.prune_low_magnitude(
    # tf.keras.models.clone_model(model),
    model,
    # pruning_schedule=tfmot.sparsity.keras.ConstantSparsity(0.5, 0)
)

print("Sparsity:", (model_for_pruning.get_weights()[0] == 0).mean())

import tempfile

log_dir = tempfile.mkdtemp()
callbacks = [
    tfmot.sparsity.keras.UpdatePruningStep(),
    # Log sparsity and other metrics in Tensorboard.
    tfmot.sparsity.keras.PruningSummaries(log_dir=log_dir)
]

model_for_pruning.compile(
      loss=tf.keras.losses.MeanSquaredError(),
      optimizer='adam',
      metrics=['accuracy']
)

model_for_pruning.fit(
    np.random.randn(100, 28, 28).astype(np.float32),
    np.random.randn(100, 10).astype(np.float32),
    callbacks=callbacks,
    epochs=2,
    batch_size=5,
    # validation_split=0.1,
)

# model_for_pruning.fit(
#     np.random.randn(1, size, size, n_channels).astype(np.float32),
#     np.random.randn(1, 1, 1, n_channels).astype(np.float32),
#     callbacks=callbacks,
#     epochs=2,
# )

# step_callback = tfmot.sparsity.keras.UpdatePruningStep()
# step_callback.set_model(model_for_pruning)
# step_callback.on_train_begin()
# step_callback.on_train_batch_begin(batch=-1)
# step_callback.on_epoch_end(batch=-1)

# model_for_pruning.summary()

print("Sparsity:", (model_for_pruning.get_weights()[1] == 0).mean())

# model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)
# model_for_export.summary()

# model_for_pruning.summary()

In [ ]:
keras_model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(28, 28)),
    tf.keras.layers.Reshape(target_shape=(28, 28, 1)),
    tf.keras.layers.Conv2D(filters=12, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10)
])

runtimes = [
    qb.TFLite(quantization_mode="off"),
    qb.ONNXRuntime(quantization_mode="off"),
    qb.TFLite(quantization_mode="off", sparsity=0.5),
    qb.TFLite(quantization_mode="off", sparsity=0.9),
    qb.TFLite(quantization_mode="off", sparsity=0.99),
    qb.TFLite(quantization_mode="static", sparsity=0.99),
]

df = qb.benchmark(keras_model, None, runtimes)

In [ ]:
sns.displot(data=df, x="inference_time", hue="name", kind="ecdf", log_scale=True)

In [ ]:
sns.boxplot(data=df, x="inference_time", y="name")

In [ ]:
import ddsp.training

model = ddsp.training.nn.DilatedConvStack(
    ch=64,
    layers_per_stack=3,
    stacks=1,
    kernel_size=3,
    dilation=2,
)
model = tf.keras.Sequential(
    [
        tf.keras.layers.Input((64000, 1, 1)),
        model,
    ]
)

model2 = tf.keras.models.clone_model(model)

In [ ]:
%config InlineBackend.print_figure_kwargs={'facecolor': "w"}

import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10, 5]

import seaborn as sns

df = pd.read_csv("/Volumes/euler/benchmark_data/0526_dense.csv")
df = df.drop(columns={"Unnamed: 0"})

g = sns.relplot(
    data=df[df["name"].str.contains("0.99")],
    hue="name",
    x="network_size",
    y="inference_time",
    kind="line",
#     ci="sd",
)
g.set(title="", yscale="log")

## Dense

In [ ]:
df = pd.read_csv("/Volumes/euler/benchmark_data/0527_dense_2.csv")
df = df.drop(columns={"Unnamed: 0"})

g = sns.relplot(
    data=df,  #df[df["name"].str.contains("0.99")],
    hue="name",
    x="network_size",
    y="inference_time",
    kind="line",
    #     ci="sd",
)
g.set(title="", yscale="log")
# sns.displot(data=df[df["network_size"] == max(df["network_size"])], x="inference_time", hue="name", kind="ecdf", log_scale=True)
#
# sns.displot(data=df[df["name"].str.contains("TFLite") & (df["network_size"] == max(df["network_size"]))],
#             x="inference_time", hue="name", kind="ecdf", log_scale=True)

In [ ]:
sns.barplot(data=df[df["network_size"] == max(df["network_size"]) - 1], y="name", x="inference_time")

In [ ]:
df2 = df.loc[df["name"].str.contains("TFLite")].copy()
df2["quantization"] = "off"
df2.loc[df2["name"].str.contains("quant_dynamic"), "quantization"] = "dynamic"
df2.loc[df2["name"].str.contains("quant_static"), "quantization"] = "static"

df2["sparsity"] = 0.0
df2.loc[df2["name"].str.contains("0.90_sparse"), "sparsity"] = 0.90
df2.loc[df2["name"].str.contains("0.99_sparse"), "sparsity"] = 0.99
# sns.heatmap(df2)
# sns.heatmap(df2.pivot("sparsity", "quantization", "inference_time"))
df2 = df2.groupby(["sparsity", "quantization"]).agg({"inference_time": "mean"}).reset_index().pivot("sparsity", "quantization", "inference_time")
sns.heatmap(df2, annot=True)

## CNN

In [ ]:
df = pd.read_csv("/Volumes/euler/benchmark_data/0527_cnn_2.csv")
df = df.drop(columns={"Unnamed: 0"})

g = sns.relplot(
    data=df,  #df[df["name"].str.contains("0.99")],
    hue="name",
    x="network_size",
    y="inference_time",
    kind="line",
    #     ci="sd",
)
g.set(title="", yscale="log")
# sns.displot(data=df[df["network_size"] == max(df["network_size"])], x="inference_time", hue="name", kind="ecdf", log_scale=True)
# sns.displot(data=df[df["name"].str.contains("TFLite") & (df["network_size"] == max(df["network_size"]))],
#             x="inference_time", hue="name", kind="ecdf", log_scale=True)
plt.show()
sns.barplot(data=df[df["network_size"] == max(df["network_size"])], y="name", x="inference_time")

## Dilated conv

In [ ]:
df = pd.read_csv("/Volumes/euler/benchmark_data/0527_dilated_conv_2.csv")
df = df.drop(columns={"Unnamed: 0"})

g = sns.relplot(
    data=df,  #df[df["name"].str.contains("0.99")],
    hue="name",
    x="network_size",
    y="inference_time",
    kind="line",
    #     ci="sd",
)
# g.set(title="", yscale="log")
# sns.displot(data=df[df["network_size"] == max(df["network_size"])], x="inference_time", hue="name", kind="ecdf", log_scale=True)
# sns.displot(data=df[df["name"].str.contains("TFLite") & (df["network_size"] == max(df["network_size"]))],
#             x="inference_time", hue="name", kind="ecdf", log_scale=True)
# plt.show()
sns.barplot(data=df[df["network_size"] == max(df["network_size"])], y="name", x="inference_time")